In [ ]:
### ConversationBufferMemory -> 전부 저장하는 무친놈
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True) ### return messages는 Chatmodel용임
memory.save_context(
    {"input": "Hi"},
    {"output": "Get off"}
)

memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi'), AIMessage(content='Get off')]}

In [7]:
### ConversationBufferWindowMemory -> 최신 몇개만 저장하는 무친놈
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(
    return_messages=True,
    k=2,
)

def add_message(input, output):
    memory.save_context(
        {"input": input},
        {"output": output}
    )

add_message("1+1", "2")
add_message("2+2", "4")

memory.load_memory_variables({})

{'history': [HumanMessage(content='1+1'),
  AIMessage(content='2'),
  HumanMessage(content='2+2'),
  AIMessage(content='4')]}

In [ ]:
### ConversationSummaryMemory -> 메세지 요약 ㄱㄴ, 하지만 초반에는 오히려 사용량이 더 많음.
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryMemory(llm=llm)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


def get_history():
    return memory.load_memory_variables({})


add_message("안녕하다, 나는 경제학을 전공중인 3학년 대학생이다.", "어쩌라는겐가...?")
add_message("요즘 경제학, Ai 분야에서 핫한걸 알려달라", "내가 알면 이러고 있겠는가...?")

get_history()

{'history': 'The human greets the AI in Korean and mentions that they are a third-year college student majoring in economics. The AI responds with confusion when the human asks about the hot trends in economics and AI.'}

In [ ]:
### ConversationSummaryBufferMemory -> 정해진 토큰 초과시 가장 오래된 메시지부터 요약시작
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=150,
    return_messages=True,
)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


def get_history():
    return memory.load_memory_variables({})

add_message("안녕하다, 나는 경제학을 전공중인 3학년 대학생이다.", "어쩌라는겐가...?")
add_message("요즘 경제학, Ai 분야에서 핫한걸 알려달라", "내가 알면 이러고 있겠는가...?")

get_history()

{'history': [HumanMessage(content='안녕하다, 나는 경제학을 전공중인 3학년 대학생이다.'),
  AIMessage(content='어쩌라는겐가...?'),
  HumanMessage(content='요즘 경제학, Ai 분야에서 핫한걸 알려달라'),
  AIMessage(content='내가 알면 이러고 있겠는가...?')]}

In [ ]:
### Conversation Knowledge Graph Memory
from langchain.memory import ConversationKGMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationKGMemory(
    llm=llm,
    return_messages=True,
)

def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


add_message("Hi, my name is John.", "Hello John! Nice to meet you 😊")
add_message("I work at Google in California.", "Wow, Google is a big company in California!")

memory.load_memory_variables({"input": "Who am I?"})

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ServiceUnavailableError: The server is overloaded or not ready yet..


{'history': []}

In [ ]:
### Memory on LLMChain (off the shelf...?)
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    return_messages=True,
    max_token_limit=80,
    memory_key="chat_history"
)

template = """
    You are a helpful AI talking to a human.

    {chat_history}
    Human:{question}
    You:
"""

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template(template),
    verbose=True, ###propmt의 로그출력
)

chain.predict(question="My name is 범준")




> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    []
    Human:My name is 범준
    You:


> Finished chain.


'Hello 범준! How can I assist you today?'

In [ ]:
chain.predict(question="Say my name agian")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    [HumanMessage(content='My name is 범준'), AIMessage(content='Hello 범준! How can I assist you today?')]
    Human:Say my name agian
    You:


> Finished chain.


'Hello 범준! How can I assist you today?'

In [ ]:
### Memory on LLMChain + ChatPromptTemplate (off the shelf...?)
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    return_messages=True,
    max_token_limit=80,
    memory_key="chat_history"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=True, ###propmt의 로그출력
)

chain.predict(question="My name is 범준")
chain.predict(question="What's my name?")





> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: My name is 범준

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: My name is 범준
AI: Hello 범준! How can I assist you today?
Human: What's my name?

> Finished chain.


'Your name is 범준.'

In [ ]:
### Memory on Langchain Expression Language(LCEL, on the shelf...?)
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    return_messages=True,
    max_token_limit=80,
    memory_key="chat_history"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

def load_memory(input):
    print(input)
    return memory.load_memory_variables({})["chat_history"]

chain = RunnablePassthrough.assign(chat_history=load_memory) | prompt | llm

def invoke_chain(question):    
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)

In [29]:
invoke_chain("My name is beomjun")

{'question': 'My name is beomjun'}
content='Hello Beomjun! How can I assist you today?'


In [30]:
invoke_chain("What is my name?")

{'question': 'What is my name?'}
content='Your name is Beomjun.'
